# 站間量分析

需要準備資料：
1. 票證資料：須包含所有佔位點的資料
2. 站序資料：需帶有'Direction'欄位
3. 班表資料：需帶有'Direction'、'IsWorkday'欄位
4. 營運月報 (optional)：做票證放大率佐證用 

## 基礎設定

包含環境設定，以及指定對應資料夾路徑（input、process、output）

In [1]:
import os
import pandas as pd
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display
from tickets_cleaning import tickets_cleaning, date_defined , getDaysCount, getMagnification, tickets_match_shift

In [2]:
inputfolder_path = os.path.join(os.getcwd(),'..', 'input')
outputfolder_path = os.path.join(os.getcwd(),'..', 'output')
processfolder_path = os.path.join(os.getcwd(),'..', 'process')

# 確保資料夾存在
os.makedirs(inputfolder_path, exist_ok=True)
os.makedirs(outputfolder_path, exist_ok=True)
os.makedirs(processfolder_path, exist_ok=True)


In [ ]:
# 加入補班、國定假日等日期  Note:需要手動調整
date_turn_holiday=[20230929] # 補假、國定假日、颱風天
date_turn_workday=[20230923] # 補班
startdate = 20230701
enddate = 20230930
# 計算每月的假日與平日數
dayscount = getDaysCount(startdate, enddate, date_turn_holiday, date_turn_workday)


In [ ]:
# 定義票證資料的欄位名稱 Note:需要手動調整
direction_col = 'DIRECTION'
getontime_col ='GETON_DATE'
getofftime_col ='GETOFF_DATE'
getonstop_col ='GETON_STOP_NAME'
getoffstop_col ='GETOFF_STOP_NAME'
getonseq_col ='GETON_STOP_SEQ'
getoffseq_col ='GETOFF_STOP_SEQ'
routename_col = "ROUTE_NAME"

## 資料前處理

1. 票證清洗(去除不可用資料)
2. 票證定義日期欄位 (年月、平假日)
3. 處理票證放大率

In [12]:
'''進行基礎的票證清洗
1. 找到上車時間 < 下車時間
2. 上車站序 < 下車站序
3. 上下車站名不同'''

# 定義 tickets.csv 的相對路徑
tickets_path = os.path.join(inputfolder_path , 'tickets.csv')
tickets = pd.read_csv(tickets_path)

# 讀取資料並進行清理
tickets, errorstat, correctrate = tickets_cleaning(tickets, 
    getontime=getontime_col, 
    getofftime=getofftime_col, 
    getonstop=getonstop_col, 
    getoffstop=getoffstop_col, 
    getonseq=getonseq_col, 
    getoffseq=getoffseq_col)

# 把清洗過的資料轉存至process
tickets.to_csv(os.path.join(processfolder_path , 'tickets_cleaned.csv'))
# tickets.to_csv(os.path.join(os.path.dirname(__file__), '..', 'process', 'tickets_cleaned.csv'))

# 輸出數據清洗統計
errorstat_path = os.path.join(outputfolder_path , 'ErrorDataStat.txt')
with open(errorstat_path , 'w', encoding='utf-8') as file:
    for key, value in errorstat.items():
        file.write(f"{key}: {value}\n")
# del errorstat
tickets = date_defined(tickets, getontime_columns=getontime_col, date_turn_holiday=date_turn_holiday,\
                       date_turn_workday=date_turn_workday)


C:\Users\kjchang\AppData\Local\Temp\ipykernel_30312\3120557935.py:8: DtypeWarning: Columns (4,5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  tickets = pd.read_csv(tickets_path)


In [ ]:
# tickets = pd.read_csv(os.path.join(processfolder_path , 'tickets_cleaned.csv'))
# tickets = tickets.rename(columns = {'RouteName':'ROUTE_NAME'})

In [15]:
'''處理票證資料放大率'''

operation = pd.read_csv(os.path.join(inputfolder_path, 'operation.csv'))

# 計算 DataYearMonth 並格式化
operation['DataYearMonth'] = (
    pd.to_datetime((operation['YEAR'] + 1911) * 100 + operation['MONTH'], format='%Y%m')
    .dt.strftime('%Y%m')
)

tickets_magnification = getMagnification(
    tickets=tickets,
    tickets_routename_col=routename_col,
    tickets_yearmonth_col='DataYearMonth',  # 指定票證數據的年月欄位 Note:需要手動調整
    operation=operation,
    operation_routename_col='ROUTE_NAME', # 這個Operation須要手動調整欄位名稱 Note:需要手動調整
    operation_yearmonth_col='DataYearMonth',  # 指定運營數據的年月欄位 Note:需要手動調整
    operation_passengers_col='PASSENGERS' # Note:需要手動調整
)

# 列出所有放大率會有異常的路線
ooc_route_list = list(set(tickets_magnification[tickets_magnification['Magnification'] >= 1.3]['RouteName'].unique()).union(
    set(tickets_magnification[tickets_magnification['Magnification'] <= 0.8]['RouteName'].unique())
))

In [17]:
# # 讀取班表資料
# shift = pd.read_excel(os.path.join(inputfolder_path, 'shift.xlsx'))
# shift.columns = ['RouteName', 'Direction', 'Shift', 'IsWorkday']
# shift['IsWorkday'] = shift['IsWorkday'].replace({'假日': '0', '平日': '1'})
# shift['Shift'] = shift['Shift'].astype(str)
# shift['Shift'] = pd.to_datetime(shift['Shift'], format='%H:%M').dt.time
# shift = shift.sort_values(['RouteName', 'IsWorkday','Shift', 'Direction'], ascending=[True, True, True, True])

# 讀取班表資料
shift = pd.read_csv(os.path.join(inputfolder_path, 'shift.csv'))
shift['Shift'] = shift['Shift'].astype(str)
shift['Shift'] = pd.to_datetime(shift['Shift'], format='%H:%M').dt.time
shift = shift.sort_values(['RouteName', 'IsWorkday','Shift', 'Direction'], ascending=[True, True, True, True]).reset_index(drop = True)

# 讀取相關的站序 
seq = pd.read_csv(os.path.join(inputfolder_path,'seq.csv'))

# 具有班表的RouteName_list
shift_routename_list = list(shift['RouteName'].unique()) 
tickets_routename_list = list(tickets[routename_col].unique())
seq_routename_list = list(shift['RouteName'].unique())

# 不在 tickets_routename_list 中但在 shift_routename_list 中的項目
only_in_shift = list(set(shift_routename_list) - set(tickets_routename_list))
# 不在 shift_routename_list 中但在 tickets_routename_list 中的項目
only_in_tickets = list(set(tickets_routename_list) - set(shift_routename_list))
common_routes = list(set(tickets_routename_list) & set(shift_routename_list) & set(seq_routename_list))

# 印出結果
print("缺票證資料:", only_in_shift)
print("缺班表資料:", only_in_tickets)
print("本次可算的路線:", common_routes)

缺票證資料: ['T513', '5050A', '5008', 'T516A', '724', '5096A', '5035', '201', '5081A', '181', '262', '723', '309C', '5096', '5011', '5109A', 'T515', '266', '261', 'T517A', 'T517', '609', 'T516B', '722', '119', '5086C', '506', '5010', '5019', '208C', '265A', '5616B', '5106A', '608', '263', '176', '5107C', '5616A', 'T512A', '5022A', '722A', '5616C', '265B', '5022', '5018', '5107A', '5006', 'T516', 'T512', '5106B', '5107B', '723A', '5099A']
缺班表資料: ['大溪高中(中壢C+E線)', '南崁高中(大園線)', '壽山高中三鶯線學生專車', '桃園高中南祥線學生專車', '5646B', 'S315', '桃/壽高中-林口線', '大園國際高中(永安大竹線)', '南崁高中-寶慶線', '南崁高中(八德大有線)', '大園高中(楊梅線)', '503', '720', '內壢高中竹圍甲線學生專車', '壽山高中大園線學生專車', '內壢高中觀音線學生專車', '桃園高中(武陵線)', '南崁高中(青溪中正線)', '大溪高中(中壢BD線)', 'F902A', '中壢高中大溪線學生專車', 602, 603, 606, 607, '內壢高中(龍岡線)', '大溪高遶經羅浮學專車', '陽明高中大園線學生專車', '大園高中中正Ａ線學生專車', '青埔國中(大江線)', '大溪高中桃園Ａ線學生專車', '505', '觀音高中永安線學生專車', '桃園高中-大溪線', '中大壢中(大湳線)', '內壢高中渴望中豐線學生專車', '永豐高中龍潭線學生專車', 703, '內壢高中(新屋線)', '大園高中士校內壢線學生專車', 708, 'F903', '永豐高中龜山線學生專車', '大溪高中蝙蝠洞線學生專車', '武陵高中-大溪線', '平鎮高中

### 基本判讀指標：是否繼續往下做

1. 列出本次資料正常資料的佔比
2. 列出本次放大率異常的路線 ( 可以進一步以plotly 圖表檢視長條圖)

In [18]:
print(f'資料可用比例 = {correctrate}%',end=' ')
if correctrate <= 95:
    print('本次取得的資料錯誤率太高，建議重新檢視')
else : 
    print('本次的資料可以使用')

try:
    if len(ooc_route_list) > 0:
        print(f'本次放大率異常路線共{len(ooc_route_list)}條')
        print('票證放大率異常的路線編號', end= ':')
        print(ooc_route_list)
except:
    pass

資料可用比例 = 98.9% 本次的資料可以使用
本次放大率異常路線共327條
票證放大率異常的路線編號:['5078', '5020', '大溪高中(中壢C+E線)', '253', '715', '桃園高中南祥線學生專車', '5646B', '113', '南崁高中(八德大有線)', '大園高中(楊梅線)', '503', '5071', '125', '720', '5057C', '壽山高中大園線學生專車', '115A', '內壢高中觀音線學生專車', 'BR', '5014', '5083', '5646', '大溪高中(中壢BD線)', '105A', '5049', 'F902A', '709', '中壢高中大溪線學生專車', '5038', '5094', '5099', '129', '5112', '5028', '5104', '213', '5057B', '大溪高中桃園Ａ線學生專車', '5623', '708', '5118', '152', '5039', '5104A', '觀音高中永安線學生專車', '5055', '5107', '中大壢中(大湳線)', '173', '5068', '5081', '171', '252A', '5623A', '大園高中士校內壢線學生專車', 'F903', '永豐高中龜山線學生專車', '5073', '139', '大溪高中蝙蝠洞線學生專車', '709B', '武陵高中-大溪線', '平鎮高中-火車站線(下課)', 217, '602', '706B', '231', '710A', '5672', '1', 'F907', '武陵高中東興下課', '116', '5014A', '5025', '120', '5623B', '168', '5021', '5050', '5063', '710', '東興國中(華勛線)', '135', '117A', '5644', '603', '桃園高中-龍潭線', '301', '5098', '711', '117', '5015', '5105', '102', '大溪高中龍潭Ｂ線學生專車', '內壢高中(東社線)', '169A', '5647', '5084', '220A', '5032', '5617A', '桃園高中崎海線學

In [8]:
# unique_year_months = tickets_magnification["DataYearMonth"].unique()
# # 創建篩選器 (Dropdown)
# dropdown = widgets.Dropdown(
#     options=unique_year_months,
#     value=unique_year_months[0],
#     description="月份:"
# )

# # 定義繪圖函數
# def plot_barchart(selected_month):
#     # 篩選 DataFrame
#     filtered_df = tickets_magnification[tickets_magnification["DataYearMonth"] == selected_month]
    
#     if filtered_df.empty:
#         print(f"No data available for {selected_month}")
#         return
    
#     # 創建條形圖
#     fig = go.Figure()

#     # 定義顯示在 hover 上的格式
#     hover_text_tickets = [
#     f"RouteName: {row['RouteName']}<br>Magnification: {row['Magnification'] * 100:.2f}%<br>Tickets: {row['Tickets']:,}"  # Magnification 顯示為百分比，Tickets 顯示為實際數字
#     for _, row in filtered_df.iterrows()
#     ]
#     hover_text_passengers = [
#     f"RouteName: {row['RouteName']}<br>Magnification: {row['Magnification'] * 100:.2f}%<br>Passengers: {row['Passengers']:,}"  # Magnification 顯示為百分比，Passengers 顯示為實際數字
#     for _, row in filtered_df.iterrows()
#     ]

#     # 添加 Tickets 的長條圖
#     fig.add_trace(go.Bar(
#         x=filtered_df["RouteName"],
#         y=filtered_df["Tickets"],
#         name="Tickets",
#         marker_color="#84C1FF",
#         hovertext=hover_text_tickets,  # 顯示格式化過的 hovertext
#         hoverinfo="text"  # 只顯示 hovertext 的內容
#     ))

#     # 添加 Passengers 的長條圖
#     fig.add_trace(go.Bar(
#         x=filtered_df["RouteName"],
#         y=filtered_df["Passengers"],  # 更新欄位名稱為 Passengers
#         name="Passengers",
#         marker_color="#FF8000",
#         hovertext=hover_text_passengers,  # 顯示格式化過的 hovertext
#         hoverinfo="text"  # 只顯示 hovertext 的內容
#     ))

#     # 設定標題與軸標籤
#     fig.update_layout(
#         title=f"Tickets and Passengers for {selected_month}",
#         xaxis_title="路線編號",
#         yaxis_title="人次",
#         barmode="group",  # 並列顯示長條圖
#         xaxis_tickangle=-90,
#         template="plotly_white"  # 使用白色背景的模板
#     )

#     # 顯示圖表
#     fig.show()

# # 綁定事件到篩選器
# dropdown.observe(lambda change: plot_barchart(change.new), names="value")

# # 初始顯示
# display(dropdown)
# plot_barchart(dropdown.value)


## 資料運算

逐條路線執行
1. 比對班次：依據上車時間比對發車班次（適用班次兼具較少的班次）
2. 班表 - 站序配對
3. 計算**上下車人次**及**站間量**

In [109]:
common_routesssssss = common_routes.copy()

In [64]:
route = common_routes[0]
print(route)

115B


In [110]:
common_routes[:3]

['115B', '5078', '5020']

In [ ]:
common_routes

In [112]:
outputfolder_unmagnification_path = os.path.join(os.getcwd(),'..', 'output','未放大過的每班次載客')
os.makedirs(outputfolder_unmagnification_path, exist_ok=True)

In [122]:
import os
import pandas as pd

# 假設變數已經初始化，例如：`common_routes`, `outputfolder_unmagnification_path`, `seq`, `shift`, `dayscount`, `tickets`

# 迴圈計算
for route in common_routes[:3]:
    print(route)
    routefilename = os.path.join(outputfolder_unmagnification_path, f'{route}.xlsx')
    
    with pd.ExcelWriter(routefilename) as writer:
        # 篩選特定路線的資料
        seq_select = seq[seq['RouteName'] == route].reset_index(drop=True)
        shift_select = shift[shift['RouteName'] == route].reset_index(drop=True)
        
        # 呼叫平假日天數
        holidaycount = dayscount[dayscount['DataYearMonth'] == yearmonth]['Holiday'].values[0]
        Workdaycount = dayscount[dayscount['DataYearMonth'] == yearmonth]['Workday'].values[0]
        shift_select['Days'] = shift_select['IsWorkday'].apply(lambda x: Workdaycount if x == 1 else holidaycount)
        
        # 合併班表和路線序列
        seqwithshift = pd.merge(shift_select, seq_select, on=['RouteName', 'Direction'])
        
        # 篩選符合月份的票證資料
        tickets_select = tickets[tickets[routename_col] == route].sort_values(getontime_col).reset_index(drop=True)
        yearmonthlist = list(tickets['DataYearMonth'].unique())
        
        onandoff_MAX_list = []

        # 針對每個月份進行處理
        for yearmonth in yearmonthlist:
            print(yearmonth)
            sheetname = str(yearmonth)
            
            # 篩選該月份的票證資料
            tickets_select_month = tickets_select[tickets_select['DataYearMonth'] == yearmonth]
            tickets_select_month = tickets_select_month[tickets_select_month[direction_col].isin(list(shift_select['Direction'].unique()))].reset_index(drop=True)
            
            # 比對班表
            tickets_select_month = tickets_match_shift(
                tickets=tickets_select_month,
                shifts=shift_select,
                routename_col=routename_col,
                getontime_col=getontime_col,
                direction_col=direction_col
            )
            
            # 計算每站上下車數據
            onandoff = pd.merge(
                seqwithshift,
                tickets_select_month.groupby([routename_col, direction_col, 'IsWorkday', getonseq_col, 'Matched_Shift'])
                .size().reset_index(name='OnCount')
                .rename(columns={routename_col: 'RouteName', direction_col: 'Direction', getonseq_col: 'Seq', 'Matched_Shift': 'Shift'}),
                on=['RouteName', 'Direction', 'IsWorkday', 'Seq', 'Shift'], how='left'
            )
            onandoff = pd.merge(
                onandoff,
                tickets_select_month.groupby([routename_col, direction_col, 'IsWorkday', getoffseq_col, 'Matched_Shift'])
                .size().reset_index(name='OffCount')
                .rename(columns={routename_col: 'RouteName', direction_col: 'Direction', getoffseq_col: 'Seq', 'Matched_Shift': 'Shift'}),
                on=['RouteName', 'Direction', 'IsWorkday', 'Seq', 'Shift'], how='left'
            )
            onandoff[['OnCount', 'OffCount']] = onandoff[['OnCount', 'OffCount']].fillna(0).astype(int)
            
            # 計算站間累計人數
            for i in range(len(onandoff)):
                if onandoff.loc[i, 'Seq'] == 1:
                    onandoff.loc[i, 'OnBus'] = onandoff.loc[i, 'OnCount'] - onandoff.loc[i, 'OffCount']
                else:
                    onandoff.loc[i, 'OnBus'] = onandoff.loc[i - 1, 'OnBus'] + onandoff.loc[i, 'OnCount'] - onandoff.loc[i, 'OffCount']
            onandoff.loc[onandoff['OnBus'] < 0, 'OnBus'] = 0
            
            # 平均上下車數
            onandoff['OnAVG'] = round(onandoff['OnCount'] / onandoff['Days'], 1)
            onandoff['OffAVG'] = round(onandoff['OffCount'] / onandoff['Days'], 1)
            
            # 平均站間人數
            for i in range(len(onandoff)):
                if onandoff.loc[i, 'Seq'] == 1:
                    onandoff.loc[i, 'OnBusAvg'] = onandoff.loc[i, 'OnAVG'] - onandoff.loc[i, 'OffAVG']
                else:
                    onandoff.loc[i, 'OnBusAvg'] = onandoff.loc[i - 1, 'OnBusAvg'] + onandoff.loc[i, 'OnAVG'] - onandoff.loc[i, 'OffAVG']
            onandoff.loc[onandoff['OnBusAvg'] < 0, 'OnBusAvg'] = 0
            
            # 寫入該月份的數據
            onandoff.to_excel(writer, sheet_name=sheetname, index=False)
            
            # 計算每月最大站間量
            onandoff_MAX = onandoff.groupby(['RouteName', 'Direction']).agg(OnBusMax=('OnBusAvg', 'max')).reset_index()
            onandoff_MAX['DataYearMonth'] = yearmonth
            onandoff_MAX_list.append(onandoff_MAX)
        
        # 整理每月最大站間量並寫入
        if onandoff_MAX_list:
            onandoff_MAX_list = pd.concat(onandoff_MAX_list)
            onandoff_MAX_list.to_excel(writer, sheet_name="每月最大站間量", index=False)


115B
202309
202308
202307
5078
202309
202308
202307
5020
202309
202308
202307


In [121]:
# 要開始用for 迴圈套入route 進行計算
for route in common_routes[:3]:
    print(route)
    routefilename = os.path.join(outputfolder_unmagnification_path,f'{route}.xlsx')
    with pd.ExcelWriter(routefilename) as writer:
    seq_select = seq[seq['RouteName'] == route].reset_index(drop = True)
    shift_select = shift[shift['RouteName'] == route].reset_index(drop = True)
    # 呼叫前面所儲存的 平假日天數
    holidaycount = dayscount[dayscount['DataYearMonth'] == yearmonth]['Holiday'].values[0]
    Workdaycount = dayscount[dayscount['DataYearMonth'] == yearmonth]['Workday'].values[0]
    shift_select['Days'] = shift_select['IsWorkday'].apply(lambda x: Workdaycount if x == 1 else holidaycount)
    seqwithshift = pd.merge(shift_select, seq_select, on = ['RouteName','Direction'])
    # 先挑選特定路線
    tickets_select = tickets[tickets[routename_col] == route].sort_values(getontime_col).reset_index(drop = True)
    
    # 根據這個路線 挑選他有提供的月份進行計算
    yearmonthlist = list(tickets['DataYearMonth'].unique())

    onandoff_MAX_list = []

    for yearmonth in yearmonthlist:
        print(yearmonth)
        sheetname = str(yearmonth)
        tickets_select_month = tickets_select[tickets_select['DataYearMonth'] == yearmonth].sort_values(getontime_col).reset_index(drop = True)
        tickets_select_month = tickets_select_month[tickets_select_month[direction_col].isin(list(shift_select['Direction'].unique()))].sort_values(getontime_col).reset_index(drop = True)

        # 比對班表
        tickets_select_month = tickets_match_shift(tickets=tickets_select_month, shifts=shift_select, routename_col=routename_col, getontime_col=getontime_col, direction_col=direction_col)

        # 計算每站每月平均上下車人次
        onandoff = pd.merge(seqwithshift, \
                            tickets_select_month.groupby([routename_col,direction_col,'IsWorkday',getonseq_col,'Matched_Shift']).size().reset_index(name = 'OnCount').rename(columns = {routename_col:'RouteName',direction_col:'Direction', getonseq_col:'Seq','Matched_Shift':'Shift'}),\
                                on = ['RouteName', 'Direction','IsWorkday','Seq','Shift'], how = 'left')
        onandoff = pd.merge(onandoff, \
                            tickets_select_month.groupby([routename_col,direction_col,'IsWorkday',getoffseq_col,'Matched_Shift']).size().reset_index(name = 'OffCount').rename(columns = {routename_col:'RouteName',direction_col:'Direction', getoffseq_col:'Seq','Matched_Shift':'Shift'}),\
                                on = ['RouteName', 'Direction','IsWorkday','Seq','Shift'], how = 'left')
        onandoff[['OnCount', 'OffCount']] = onandoff[['OnCount', 'OffCount']].fillna(0).astype('int64')

        # 計算站間量
        for i in range(len(onandoff)):
            if onandoff.loc[i, 'Seq'] == 1:
                onandoff.loc[i, 'OnBus'] = onandoff.loc[i, 'OnCount'] - onandoff.loc[i, 'OffCount']
            else:
                if i > 0:
                    onandoff.loc[i, 'OnBus'] = onandoff.loc[i-1, 'OnBus'] + onandoff.loc[i, 'OnCount'] - onandoff.loc[i, 'OffCount']
                else:
                    # 如果 i == 0，沒有前一個值，所以我們只使用當前的OnCount和OffCount
                    onandoff.loc[i, 'OnBus'] = onandoff.loc[i, 'OnCount'] - onandoff.loc[i, 'OffCount']
        onandoff.loc[onandoff['OnBus'] < 0, 'OnBus'] = 0

        onandoff['OnAVG'] = round(onandoff['OnCount'] / onandoff['Days'],1)
        onandoff['OffAVG'] = round(onandoff['OffCount'] / onandoff['Days'],1)

        # 計算站間量
        for i in range(len(onandoff)):
            if onandoff.loc[i, 'Seq'] == 1:
                onandoff.loc[i, 'OnBusAvg'] = onandoff.loc[i, 'OnAVG'] - onandoff.loc[i, 'OffAVG']
            else:
                if i > 0:
                    onandoff.loc[i, 'OnBusAvg'] = onandoff.loc[i-1, 'OnBusAvg'] + onandoff.loc[i, 'OnAVG'] - onandoff.loc[i, 'OffAVG']
                else:
                    # 如果 i == 0，沒有前一個值，所以我們只使用當前的OnAVG和OffAVG
                    onandoff.loc[i, 'OnBusAvg'] = onandoff.loc[i, 'OnAVG'] - onandoff.loc[i, 'OffAVG']
        onandoff.loc[onandoff['OnBusAvg'] < 0, 'OnBusAvg'] = 0
        onandoff.to_excel(writer, sheet_name=sheetname, index=False)

        onandoff_MAX = onandoff.groupby(['RouteName', 'Direction']).agg(OnBusMax=('OnBusAvg', 'max')).reset_index() # 每個班次的最大站間量
        onandoff_MAX['DataYearMonth'] = yearmonth

        if yearmonth == yearmonthlist[-1]: 
            onandoff_MAX_list = onandoff_MAX_list.append(onandoff_MAX)
            onandoff_MAX_list = pd.concat(onandoff_MAX_list)
            onandoff_MAX_list.to_excel(writer, sheet_name="每月最大站間量", index=False)
            


IndentationError: expected an indented block after 'with' statement on line 5 (3678801289.py, line 6)

In [120]:
onandoff_MAX

,RouteName,Direction,Shift,OnBusMax
0,5020,0,06:37:00,21.8
1,5020,0,08:40:00,12.9
2,5020,0,10:30:00,15.3
3,5020,0,10:40:00,22.4
4,5020,0,14:30:00,8.1
5,5020,0,16:10:00,27.6
6,5020,0,20:00:00,12.5
7,5020,1,05:45:00,17.4
8,5020,1,08:05:00,26.5
9,5020,1,10:00:00,15.6


In [108]:
seq_select = seq[seq['RouteName'] == route].reset_index(drop = True)
shift_select = shift[shift['RouteName'] == route].reset_index(drop = True)
# 呼叫前面所儲存的 平假日天數
holidaycount = dayscount[dayscount['DataYearMonth'] == yearmonth]['Holiday'].values[0]
Workdaycount = dayscount[dayscount['DataYearMonth'] == yearmonth]['Workday'].values[0]
shift_select['Days'] = shift_select['IsWorkday'].apply(lambda x: Workdaycount if x == 1 else holidaycount)

seqwithshift = pd.merge(shift_select, seq_select, on = ['RouteName','Direction'])

# 先挑選特定路線
tickets_select = tickets[tickets[routename_col] == route].sort_values(getontime_col).reset_index(drop = True)

# 根據這個路線 挑選他有提供的月份進行計算
yearmonthlist = list(tickets['DataYearMonth'].unique())
yearmonth = yearmonthlist[0]

tickets_select_month = tickets_select[tickets_select['DataYearMonth'] == yearmonth].sort_values(getontime_col).reset_index(drop = True)
tickets_select_month = tickets_select_month[tickets_select_month[direction_col].isin(list(shift_select['Direction'].unique()))].sort_values(getontime_col).reset_index(drop = True)

# 比對班表
tickets_select_month = tickets_match_shift(tickets=tickets_select_month, shifts=shift_select, routename_col=routename_col, getontime_col=getontime_col, direction_col=direction_col)

# 計算每站每月平均上下車人次
onandoff = pd.merge(seqwithshift, \
                    tickets_select_month.groupby([routename_col,direction_col,'IsWorkday',getonseq_col,'Matched_Shift']).size().reset_index(name = 'OnCount').rename(columns = {routename_col:'RouteName',direction_col:'Direction', getonseq_col:'Seq','Matched_Shift':'Shift'}),\
                        on = ['RouteName', 'Direction','IsWorkday','Seq','Shift'], how = 'left')
onandoff = pd.merge(onandoff, \
                    tickets_select_month.groupby([routename_col,direction_col,'IsWorkday',getoffseq_col,'Matched_Shift']).size().reset_index(name = 'OffCount').rename(columns = {routename_col:'RouteName',direction_col:'Direction', getoffseq_col:'Seq','Matched_Shift':'Shift'}),\
                        on = ['RouteName', 'Direction','IsWorkday','Seq','Shift'], how = 'left')
onandoff[['OnCount', 'OffCount']] = onandoff[['OnCount', 'OffCount']].fillna(0).astype('int64')

# 計算站間量
for i in range(len(onandoff)):
    if onandoff.loc[i, 'Seq'] == 1:
        onandoff.loc[i, 'OnBus'] = onandoff.loc[i, 'OnCount'] - onandoff.loc[i, 'OffCount']
    else:
        if i > 0:
            onandoff.loc[i, 'OnBus'] = onandoff.loc[i-1, 'OnBus'] + onandoff.loc[i, 'OnCount'] - onandoff.loc[i, 'OffCount']
        else:
            # 如果 i == 0，沒有前一個值，所以我們只使用當前的OnCount和OffCount
            onandoff.loc[i, 'OnBus'] = onandoff.loc[i, 'OnCount'] - onandoff.loc[i, 'OffCount']
onandoff.loc[onandoff['OnBus'] < 0, 'OnBus'] = 0

onandoff['OnAVG'] = round(onandoff['OnCount'] / onandoff['Days'],1)
onandoff['OffAVG'] = round(onandoff['OffCount'] / onandoff['Days'],1)

# 計算站間量
for i in range(len(onandoff)):
    if onandoff.loc[i, 'Seq'] == 1:
        onandoff.loc[i, 'OnBusAvg'] = onandoff.loc[i, 'OnAVG'] - onandoff.loc[i, 'OffAVG']
    else:
        if i > 0:
            onandoff.loc[i, 'OnBusAvg'] = onandoff.loc[i-1, 'OnBusAvg'] + onandoff.loc[i, 'OnAVG'] - onandoff.loc[i, 'OffAVG']
        else:
            # 如果 i == 0，沒有前一個值，所以我們只使用當前的OnAVG和OffAVG
            onandoff.loc[i, 'OnBusAvg'] = onandoff.loc[i, 'OnAVG'] - onandoff.loc[i, 'OffAVG']
onandoff.loc[onandoff['OnBusAvg'] < 0, 'OnBusAvg'] = 0
onandoff_MAX = onandoff.groupby(['RouteName', 'Direction', 'Shift']).agg(OnBusMax=('OnBusAvg', 'max')).reset_index() # 每個班次的最大站間量


'202307'

In [106]:
onandoff.groupby(['RouteName','Direction' ,'Shift']).agg({'OnBusAvg':'max'}).reset_index() #.rename(columns  = {'OnbusAvg':'OnBusMax'})

,RouteName,Direction,Shift,OnBusAvg
0,115B,0,08:00:00,7.0
1,115B,0,09:30:00,7.1
2,115B,0,10:30:00,5.3
3,115B,0,11:30:00,7.7
4,115B,0,14:00:00,5.6
5,115B,0,15:00:00,18.0
6,115B,0,17:30:00,12.6
7,115B,0,18:30:00,12.6
8,115B,0,20:00:00,14.2


以下嘗試

In [ ]:

shift_select
# # 新增 Days 欄位


,RouteName,IsWorkday,Direction,Shift,IsCycle,Days
0,115B,0,0,08:00:00,0,9
1,115B,0,0,09:30:00,0,9
2,115B,0,0,10:30:00,0,9
3,115B,0,0,11:30:00,0,9
4,115B,0,0,14:00:00,0,9
5,115B,0,0,15:00:00,0,9
6,115B,0,0,17:30:00,0,9
7,115B,0,0,18:30:00,0,9
8,115B,0,0,20:00:00,0,9
9,115B,1,0,08:00:00,0,21


,RouteName,IsWorkday,Direction,Shift,IsCycle,Days
0,115B,0,0,08:00:00,0,9
1,115B,0,0,09:30:00,0,9
2,115B,0,0,10:30:00,0,9
3,115B,0,0,11:30:00,0,9
4,115B,0,0,14:00:00,0,9
5,115B,0,0,15:00:00,0,9
6,115B,0,0,17:30:00,0,9
7,115B,0,0,18:30:00,0,9
8,115B,0,0,20:00:00,0,9
9,115B,1,0,08:00:00,0,21


In [87]:
Workdaycount

21

In [ ]:
shift_select['Days']

,RouteName,IsWorkday,Direction,Shift,IsCycle
0,115B,0,0,08:00:00,0
1,115B,0,0,09:30:00,0
2,115B,0,0,10:30:00,0
3,115B,0,0,11:30:00,0
4,115B,0,0,14:00:00,0
5,115B,0,0,15:00:00,0
6,115B,0,0,17:30:00,0
7,115B,0,0,18:30:00,0
8,115B,0,0,20:00:00,0
9,115B,1,0,08:00:00,0


In [ ]:
shift_select

In [ ]:
pd.merge(shift_select)

,RouteName,IsWorkday,Direction,Shift,IsCycle
0,115B,0,0,08:00:00,0
1,115B,0,0,09:30:00,0
2,115B,0,0,10:30:00,0
3,115B,0,0,11:30:00,0
4,115B,0,0,14:00:00,0
5,115B,0,0,15:00:00,0
6,115B,0,0,17:30:00,0
7,115B,0,0,18:30:00,0
8,115B,0,0,20:00:00,0
9,115B,1,0,08:00:00,0


In [ ]:
for i in range(len(seq)):
    if seq.loc[i, '停靠站順序'] == 1:
        seq.loc[i, '站間量'] = seq.loc[i, '上車人次'] - seq.loc[i, '下車人次']
    else:
        if i > 0:
            seq.loc[i, '站間量'] = seq.loc[i-1, '站間量'] + seq.loc[i, '上車人次'] - seq.loc[i, '下車人次']
        else:
            # 如果 i == 0，沒有前一個值，所以我們只使用當前的上車人次和下車人次
            seq.loc[i, '站間量'] = seq.loc[i, '上車人次'] - seq.loc[i, '下車人次']
seq.loc[seq['站間量'] < 0, '站間量'] = 0

In [ ]:
for i in range(len(seq)):
    if seq.loc[i, '停靠站順序'] == 1:
        seq.loc[i, '站間量'] = seq.loc[i, '上車人次'] - seq.loc[i, '下車人次']
    else:
        if i > 0:
            seq.loc[i, '站間量'] = seq.loc[i-1, '站間量'] + seq.loc[i, '上車人次'] - seq.loc[i, '下車人次']
        else:
            # 如果 i == 0，沒有前一個值，所以我們只使用當前的上車人次和下車人次
            seq.loc[i, '站間量'] = seq.loc[i, '上車人次'] - seq.loc[i, '下車人次']
seq.loc[seq['站間量'] < 0, '站間量'] = 0

,RouteName,IsWorkday,Direction,Shift,IsCycle,StopUID,StopName,Seq,Lat,Lon
0,206D,0,0,05:50:00,0,TAO6547,桃園總站,1,24.986580,121.313968
1,206D,0,0,05:50:00,0,TAO3460,建國路口,2,24.985320,121.310987
2,206D,0,0,05:50:00,0,TAO3462,桃花園飯店,3,24.989795,121.310955
3,206D,0,0,05:50:00,0,TAO1754,桃園郵局,4,24.993104,121.311965
4,206D,0,0,05:50:00,0,TAO430,永和市場,5,24.997334,121.309302
...,...,...,...,...,...,...,...,...,...,...
4731,206D,1,1,23:50:00,0,TAO2615,博愛路口,33,24.994076,121.308063
4732,206D,1,1,23:50:00,0,TAO2467,民族路口,34,24.990925,121.309563
4733,206D,1,1,23:50:00,0,TAO5328,桃花園飯店,35,24.989863,121.310936
4734,206D,1,1,23:50:00,0,TAO3461,建國路口,36,24.985282,121.311122


In [29]:
shift_select

,RouteName,IsWorkday,Direction,Shift,IsCycle
0,206D,0,0,05:50:00,0
1,206D,0,0,06:00:00,0
2,206D,0,0,06:40:00,0
3,206D,0,1,07:00:00,0
4,206D,0,0,07:30:00,0
...,...,...,...,...,...
123,206D,1,1,21:50:00,0
124,206D,1,0,22:30:00,0
125,206D,1,1,22:40:00,0
126,206D,1,1,23:10:00,0


In [28]:
seq_select

,RouteName,Direction,StopUID,StopName,Seq,Lat,Lon
0,206D,0,TAO6547,桃園總站,1,24.986580,121.313968
1,206D,0,TAO3460,建國路口,2,24.985320,121.310987
2,206D,0,TAO3462,桃花園飯店,3,24.989795,121.310955
3,206D,0,TAO1754,桃園郵局,4,24.993104,121.311965
4,206D,0,TAO430,永和市場,5,24.997334,121.309302
...,...,...,...,...,...,...,...
69,206D,1,TAO2615,博愛路口,33,24.994076,121.308063
70,206D,1,TAO2467,民族路口,34,24.990925,121.309563
71,206D,1,TAO5328,桃花園飯店,35,24.989863,121.310936
72,206D,1,TAO3461,建國路口,36,24.985282,121.311122
